*Ce notebook de David Chamont est distribué par Devlog sous licence Creative Commons - Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions. La description complète de la license est disponible à l'adresse web http://creativecommons.org/licenses/by-nc-sa/4.0/.*

# Python Devlog - Objets 3/6 : les objets composés

## Généralités

Un objet peut contenir des objets, qui peuvent contenir des objets, qui…
On peut descendre dans les niveaux successifs en enchainant les "." .

In [4]:
class Vecteur(object): pass
class Impulsion(object) : pass

imp = Impulsion()
imp.vitesse = Vecteur()
imp.vitesse.x = 1.
imp.vitesse.y = 0.
imp.vitesse.z = 0.
imp.masse = 10.

print ( imp.vitesse.x, imp.vitesse.y, imp.vitesse.z, imp.masse )

(1.0, 0.0, 0.0, 10.0)


<img src="img-objets/composition/composition.png" width=307px>

## (Non)Chainage des constructeurs

Dans un constructeur, il n'y a pas d'appel automatique aux constructeurs des sous-parties de l'objet courant, comme cela existe dans d'autres langages. Un objet n'a pas d'obligation de contenir des données, et donc tout ce qui est ajouté doit l'être explicitement, y compris dans les constructeurs.

In [7]:
class Vecteur(object):
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0
    def __str__(self):
        return "%d|%d|%d" % ( self.x, self.y, self.z )
        
class Impulsion(object):
    def __init__(self):
        self.vitesse = Vecteur()
        self.masse = 0
    def __str__(self):
        return "%s|%d" % ( self.vitesse.__str__(), self.masse )

i1 = Impulsion()
print "(%s)" % i1

(0|0|0|0)


Du point de vue de l'affectation, de la copie, de la comparaison, et de beaucoup d'autres opérations, un objet se comporte essentiellement comme un dictionnaire contenant des attributs.

<img src="img-objets/composition/composition-construction.png" width=355px>

## Affectation et copie

Lorsqu'on affecte un objet à une variable, il n'y a pas de copie. Comme d'habitude, cela génère une nouvelle variable qui référence le même objet original. Dans l'exemple ci-dessous, on voit que si je modifie un attribut de `i2`, `i1` est également modifié.

In [8]:
i2 = i1
i2.masse = 1
print "(%s), (%s)" % ( i2, i1 )

(0|0|0|1), (0|0|0|1)


<img src="img-objets/composition/composition-affectation.png" width=328px>

Pour dupliquer un objet, on peut avoir recours au module `copy`, et à sa fonction `copy`. Ainsi, si je modifie maintenant un attribut de `i2`, l'attribut correspondant de `i1` est inchangé.

In [9]:
import copy
i2 = copy.copy(i1)
i2.masse = 2
print "(%s), (%s)" % ( i2, i1 )

(0|0|0|2), (0|0|0|1)


Mais il s'agit d'une copie de surface. L'équivalent d'une affectation des attributs un à un :

In [4]:
i2 = Impulsion()
i2.vitesse = i1.vitesse # i2 = copy.copy(i1)
i2.masse = i1.masse     # i2 = copy.copy(i1)
i2.masse = 2
print "(%s), (%s)" % ( i2, i1 )

(0|0|0|2), (0|0|0|1)


<img src="img-objets/composition/composition-copie.png" width=328px>

Comme au début, pour `i2 = i1`, l'instruction `i2.vitesse = i1.vitesse` ne duplique pas l'instance de `Vecteur`, mais fait en sorte que `i2.vitesse` et `i1.vitesse` désigne la même instance. Ainsi, si on change un attribut de `i2.vitesse`, on agit également sur `i1.vitesse` :

In [5]:
i2.vitesse.z = 2
print "(%s), (%s)" % ( i2, i1 )

(0|0|2|2), (0|0|2|1)


Si on souhaite dupliquer intégralement un objet et tous ses attributs à tous les niveaux, on préfèrera la fonction `deepcopy`. Dans l'exemple suivant, on modifie un sous-sous-attribut de `i2`, sans affecter `i1`.

In [6]:
i2 = copy.deepcopy(i1)
i2.vitesse.y = 3
print "(%s), (%s)" % ( i2, i1 )

(0|3|2|1), (0|0|2|1)


<img src="img-objets/composition/composition-copie-profonde.png" width=307px>

## Egalité et identité

Malheureusement, à la différence d'un dictionnaire, l'opérateur de comparaison `==` n'est pas capable de vérifier récursivement l'égalité de tous les données membres de l'objet. A la place, il se contente de vérifier que les deux variables désignent le même objet, à la façon d'un `is`. Pour obtenir le comportement "logique", il faut par exemple redéfinir les opérateurs `__eq__`et `__ne__` à tous les niveaux :

In [16]:
class Vecteur(object):
    def __init__(self,x=0,y=0,z=0):
        self.__x, self.__y, self.__z = x, y, z
    def __str__(self):
        return "%d|%d|%d" % ( self.__x, self.__y, self.__z )
        
class Impulsion(object):
    def __init__(self,x=0,y=0,z=0,m=0):
        self.__vitesse = Vecteur(x,y,z)
        self.__masse = m
    def __str__(self):
        return "%s|%d" % ( self.__vitesse.__str__(), self.__masse )

i1 = Impulsion(1,2,3,4)

import copy
i2 = copy.deepcopy(i1)

(i1 is i2, i1==i2, i1!=i2)

(False, False, True)

In [14]:
class Vecteur(object):
    def __init__(self,x=0,y=0,z=0):
        self.__x = x
        self.__y = y
        self.__z = z
    def __str__(self):
        return "%d|%d|%d" % ( self.__x, self.__y, self.__z )
    def __eq__(self,other):
        return ( self.__x==other.__x ) and ( self.__y==other.__y ) and ( self.__z==other.__z )
        
class Impulsion(object):
    def __init__(self,x=0,y=0,z=0,m=0):
        self.__vitesse = Vecteur(x,y,z)
        self.__masse = m
    def __str__(self):
        return "%s|%d" % ( self.__vitesse.__str__(), self.__masse )
    def __eq__(self,other): 
        return ( self.__vitesse==other.__vitesse ) and ( self.__masse==other.__masse )

i1 = Impulsion(1,2,3,4)

import copy
i2 = copy.deepcopy(i1)

(i1 is i2, i1==i2, i1!=i2)

(False, True, True)

In [17]:
class Vecteur:
    def __init__(self,x=0,y=0,z=0):
        self.__x = x
        self.__y = y
        self.__z = z
    def __str__(self):
        return "%d|%d|%d" % ( self.__x, self.__y, self.__z )
    def __eq__(self,other):
        return ( self.__x==other.__x ) and ( self.__y==other.__y ) and ( self.__z==other.__z )
    def __ne__(self,other): return not(self==other)
        
class Impulsion:
    def __init__(self,x=0,y=0,z=0,m=0):
        self.__vitesse = Vecteur(x,y,z)
        self.__masse = m
    def __str__(self):
        return "%s|%d" % ( self.__vitesse.__str__(), self.__masse )
    def __eq__(self,other): 
        return ( self.__vitesse==other.__vitesse ) and ( self.__masse==other.__masse )
    def __ne__(self,other): return not(self==other)

i1 = Impulsion(1,2,3,4)

import copy
i2 = copy.deepcopy(i1)

(i1 is i2, i1==i2, i1!=i2)

(False, True, False)

*Ce notebook de David Chamont est distribué par Devlog sous licence Creative Commons - Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions. La description complète de la license est disponible à l'adresse web http://creativecommons.org/licenses/by-nc-sa/4.0/.
Travail initié en 2014 dans le cadre d'une série de formations Python organisées par le réseau Devlog, avec relectures de Nicolas Can, Sekou Diakite, Loic Gouarin et Christophe Halgand.*